In [1]:
#!pip3 install numpy     #### Do not run this cell if you already have the packages #####
#!pip3 install pandas
#!pip3 install beautifulsoup4
#!pip3 install Selenium
#!pip3 install fastf1
#!pip3 install webdriver-manager

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.firefox.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import fastf1

In [3]:
import os
import re
from datetime import date
import requests
import datetime

In [4]:
opts = webdriver.ChromeOptions() #this line WILL change for your setup
opts.add_argument("--disable-blink-features")
opts.add_argument("disable-blink-features=AutomationControlled") 
opts.add_argument("--window-size=1920,969")
opts.add_argument("--headless")
opts.add_argument('--no-sandbox')
opts.add_argument("--incognito")
opts.add_argument("--start-maximized")
opts.add_argument("--ignore-certificate-errors")
opts.add_argument("--allow-running-insecure-content")
opts.add_argument("--disable-notifications")
opts.add_argument("--disable-extensions")
opts.add_argument("--disable-infobars")
opts.add_argument("--disable-gpu")
opts.add_argument('--disable-dev-shm-usage')
opts.add_argument('--disable')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = opts);



logger         INFO 	

====== WebDriver manager ======
logger         INFO 	====== WebDriver manager ======
Current google-chrome version is 101.0.4951
logger         INFO 	Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
logger         INFO 	Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_mac64.zip
logger         INFO 	Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_mac64.zip
Driver has been saved in cache [/Users/ardaakman/.wdm/drivers/chromedriver/mac64/101.0.4951.41]
logger         INFO 	Driver has been saved in cache [/Users/ardaakman/.wdm/drivers/chromedriver/mac64/101.0.4951.41]


In [5]:
#Setting up the cache for fast API
fastf1.Cache.enable_cache('./FastF1Cache')

In [63]:
#This block is for getting all the event schedule data - not necessary to run every time, Ill make an update function instead for recent events
#Clean event schedule
def get_schedule():
    general_data = fastf1.get_event_schedule(1950)
    for year in range(1951, int(datetime.date.today().year)):
        general_data = pd.concat([general_data, fastf1.get_event_schedule(year)])
    general_data = general_data.reset_index(drop = True)
    general_data['Year'] = general_data['EventDate'].apply(lambda cur: cur.year)
    return general_data

#get requested year
def get_year(all_schedule, year):
    requested = all_schedule.loc[all_schedule['Year'] == year].reset_index(drop = True)
    return requested

#Saves all the year data, as csvs, into the csv_data directory. Furthermore, does so in the format "f1_(year).csv"
def save_all_years(all_schedule):
    for year in all_schedule['Year'].unique():
        years_schedule = get_year(all_schedule, year)
        years_schedule.to_csv("csv_data/f1_{}.csv".format(year))
    print("All Files Saved - Success!")
        

In [60]:
%ls

FastF1Cache/        csv_data/           f1_2021_data.ipynb


In [64]:
#Here, an example of how to get information about a certain year from the saved dataframe. Not you can also use fastf1 and fetch from cache, 
# but the data you get here is more 'cleaned' (index is fixed, etc.)
#Round number of 0 denotes pre-season testing
#schedule = get_schedule()
#most_recent = get_year(schedule, 2021)
#most_recent
save_all_years(schedule)

All Files Saved - Success!


In [ ]:
test_sesh = fastf1.get_session(2021, 1, 'R')
test_sesh.load()
ver_laps = test_sesh.laps.pick_driver("VER")
ver_laps

In [9]:
trial_lap = ver_laps[ver_laps['LapNumber']== 1.0]
trial_lap = trial_lap.loc[56]
trial_data = trial_lap.get_car_data()
trial_data

,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime
0,2021-03-28 15:07:10.167,10210,0,1,17,False,1,car,0 days 00:00:00.187000,0 days 00:37:10.157000
1,2021-03-28 15:07:10.407,9425,0,1,18,False,1,car,0 days 00:00:00.427000,0 days 00:37:10.397000
2,2021-03-28 15:07:10.647,7745,6,1,18,False,1,car,0 days 00:00:00.667000,0 days 00:37:10.637000
3,2021-03-28 15:07:10.887,6065,17,1,20,False,1,car,0 days 00:00:00.907000,0 days 00:37:10.877000
4,2021-03-28 15:07:11.127,4547,24,1,21,False,1,car,0 days 00:00:01.147000,0 days 00:37:11.117000
...,...,...,...,...,...,...,...,...,...,...
486,2021-03-28 15:09:07.407,11400,219,5,76,False,1,car,0 days 00:01:57.427000,0 days 00:39:07.397000
487,2021-03-28 15:09:07.647,11585,222,5,78,False,1,car,0 days 00:01:57.667000,0 days 00:39:07.637000
488,2021-03-28 15:09:07.886,11190,223,5,78,False,1,car,0 days 00:01:57.906000,0 days 00:39:07.876000
489,2021-03-28 15:09:08.087,10167,226,6,76,False,1,car,0 days 00:01:58.107000,0 days 00:39:08.077000


In [2]:
trial_telemetry = trial_lap.get_telemetry()
trial_telemetry[trial_telemetry['Brake'] == True]

NameError: name 'trial_lap' is not defined